In [4]:
# this should be run at the beginning of every notebook
%run "~/Recidiviz/pulse-data/recidiviz/tools/analyst/notebook_utils.py"

# this should be run if you are using the python cohort functions
%run "~/Recidiviz/pulse-data/recidiviz/tools/analyst/cohort_methods.py"

# this should be run if you are using the looker to jupyter functionality
%run '~/Recidiviz/recidiviz-research/utils/looker_dashboard_manager.py'


CONFIG_PATH = "/Users/jazz/Recidiviz/looker.ini"
PROJECT_ID = "recidiviz-123"

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# PROGRAMMING CAPACITY / ENROLLMENT

## COLORADO

In [24]:
QUERY = """
SELECT * FROM `recidiviz-staging.us_co_raw_data.eomis_sentencecompute` LIMIT 100000
"""

co_sentencecompute = pd.read_gbq(
    query=QUERY,
    project_id=PROJECT_ID,
    use_bqstorage_api=True,
    progress_bar_type="tqdm_notebook",
)

Downloading:   0%|          | 0/100000 [00:00<?, ?rows/s]

In [25]:
co_sentencecompute.head()

,OFFENDERID,INCARCERATIONBEGINDT,CHAINSEQUENCE,SENTENCESEQUENCE,TIMELINETYPE,SENTENCESTATUSFLAG,TIMESTARTDATE,MAXIMUMTERM,MAXSENTDAYSMR,CNTYJAILTIME,CNTYJAILTIMEMAX,CNTYJAILCREDIT,CNTYJAILCREDITMAX,SENTENCEREDUCTIONMAX,SENTENCEREDUCTIONLOSTMAX,NULLTIMEBEFOREMR,MANDATORYRELEASEDATECTRL,STATUTORYGOODTIME,TRUSTYTIME,TRUSTYTIMELOST,EARNEDTIMEVESTED,EARNEDTIMEDISASTERRELIEFVESTED,EARNEDTIMEACHIEVEMENTSVESTED,EARNEDTIMEEXCEPCONDUCTVESTED,EARNEDTIMEPROJECTED,PROVEARNEDTIMEPROJECTED,MANDATORYRELEASEDATEACTUAL,EARNEDTIME,EARNEDTIMEDISASTERRELIEF,EARNEDTIMEACHIEVEMENTS,EARNEDTIMEEXCEPTIONALCONDUCT,NULLTIMEACTIVEMR,MANDATORYRELEASEDATEESTIMATED,EARNEDTIMEPOSSIBLEMONTHS,EARNEDTIMEPOSSIBLE,MANDATORYRELEASEDATEPOSSIBLE,EARNEDTIMEPOTENTIALMONTHS,EARNEDTIMEPOTENTIAL,MANDATORYRELEASEDATEPOTENTIAL,EARNEDRELEASETIME,EARNEDRELEASEDATE,MRRULINGINDICATOR,TIMESERVEDTO,TIMEREMAININGTOMR,NETETBEFOREMR,SENTENCEBEGINDTPE,SENTENCEREDUCTION,SENTENCEREDUCTIONLOSTVESTED,NULLTIMEBEFOREPE,PAROLEELIGIBILITYDATECTRL,...,NULLTIMEACTIVEPE,PAROLEELIGIBILITYDATEESTIMATED,EARNEDTIMEPOSSIBLEMONTHSPE,EARNEDTIMEPOSSIBLEPE,PAROLEELIGIBILITYDATEPOSSIBLE,EARNEDTIMEPOTENTIALMONTHSPE,EARNEDTIMEPOTENTIALPE,PAROLEELIGIBILITYDATEPOTENTIAL,PERULINGINDICATOR,TIMEGAPEFFECTSTHISPED,PETIMESERVED,TIMEREMAININGTOPE,NETETBEFOREPE,MINIMUMTERM,MINSENTDAYS,STATUTORYGOODTIMEMIN,TRUSTYTIMEMIN,TRUSTYTIMELOSTMIN,SCHEDSUPERVTERMDT,SENTENCEENDDATE,PAROLEREVOKEDFLAG,LASTRULSENTSEQNO,DATELASTUPDATE,TIMELASTUPDATE,STAFFLASTUPDATE,OLDMRDATE,OLDPEDATE,PAROLETERMINATIONDATECTRL,PAROLETERMINATIONDATEACTUAL,PAROLETERMINATIONDATEESTIMATED,MINIMUMSUPVENDDATE,SENTENCEDISCHARGEDATEACTUAL,SENTENCEDISCHARGEDATEESTIMATED,SENTENCELAWCODE,OCSEARLIESTSTARTDATE,OCSINITIALLENGTH,OCSPSCC,OCSPSCCGOODTIME,OCSPRECONTROLEARNEDTIME,MINIMUMTOSERVEPERCENTAGE,MINTOSERVEPERCEARLIEST,SENTREDUCTIONEARLIEST,OCSNULLTIME,EARNEDTIMEEMERGRELIEFVESTED,EARNEDTIMEEMERGRELIEF,EARNEDTIMEEMERGRLFVESTEDPE,EARNEDTIMEEMERGRELIEFPE,file_id,update_datetime,is_deleted
0,10122235,2010-08-24,004,001,2,1,2018-05-22,0,0,0,0,0,0,0,0,0,1000-01-01,0,0,0,0,0,0,0,0,0,1000-01-01,0,0,0,0,0,1000-01-01,0,0,1000-01-01,0,0,1000-01-01,0,1000-01-01,None,0,0,0,2010-08-23,0,0,0,1000-01-01,...,0,1000-01-01,0,0,1000-01-01,0,0,1000-01-01,None,None,0,0,0,0,0,0,0,0,1000-01-01,1000-01-01,N,001,2022-02-28,13:03:54,23102,1000-01-01,1000-01-01,2018-05-22,2018-05-22,2018-05-22,1000-01-01,2018-05-22,2018-05-22,R0,2010-08-23,0,383,383,0,50,50,383,0,0,0,0,0,139055,2022-07-08 03:35:03.421242,False
1,10147870,2018-04-12,001,001,2,2,2021-09-23,0,0,0,0,0,0,0,0,0,1000-01-01,0,0,0,0,0,0,0,0,0,1000-01-01,0,0,0,0,0,1000-01-01,0,0,1000-01-01,0,0,1000-01-01,0,1000-01-01,None,0,0,0,2021-03-04,0,0,0,1000-01-01,...,0,1000-01-01,0,0,1000-01-01,0,0,1000-01-01,None,None,0,0,0,0,0,0,0,0,1000-01-01,1000-01-01,N,001,2022-06-06,08:33:30,22816,1000-01-01,1000-01-01,2021-09-23,2021-09-23,2021-09-23,1000-01-01,2021-09-23,2021-09-23,B,2019-10-28,0,0,0,123,50,50,243,0,0,0,0,0,139055,2022-07-08 03:35:03.421242,False
2,10070257,2020-11-03,003,001,2,2,2021-03-25,0,0,0,0,0,0,0,0,0,1000-01-01,0,0,0,0,0,0,0,0,0,1000-01-01,0,0,0,0,0,1000-01-01,0,0,1000-01-01,0,0,1000-01-01,0,1000-01-01,None,0,0,0,2020-10-30,0,0,4,1000-01-01,...,0,1000-01-01,0,0,1000-01-01,0,0,1000-01-01,None,None,0,0,0,0,0,0,0,0,1000-01-01,1000-01-01,N,001,2022-05-09,10:43:24,37470,1000-01-01,1000-01-01,2021-03-25,2021-03-25,2021-03-25,1000-01-01,2021-03-25,2021-03-25,B,2020-10-30,0,142,142,0,50,50,142,0,0,0,0,0,139055,2022-07-08 03:35:03.421242,False
3,10100678,2002-08-22,003,001,2,2,2020-04-24,0,0,0,0,0,0,0,0,0,1000-01-01,0,0,0,0,0,0,0,0,0,1000-01-01,0,0,0,0,0,1000-01-01,0,0,1000-01-01,0,0,1000-01-01,0,1000-01-01,None,0,0,0,1000-01-01,0,90,154,1000-01-01,...,0,1000-01-01,0,0,1000-01-01,0,0,1000-01-01,None,None,0,0,0,0,0,0,0,0,1000-01-01,1000-01-01,N,001,2022-06-15,12:54:31,34571,1000-01-01,1000-01-01,2020-04-24,2020-04-24,2020-04-24,1000-01-01,2020-04-24,2020-04-24,B,1000-01-01,0,0,0,0,50,50,248,0,0,0,0,0,139055,2022-07-08 03:35:03.421

In [34]:
co_sentencecompute.EARNEDTIME.astype(int).clip(upper=365).value_counts()

0      63382
10      5739
20      3247
12      3131
30      2284
       ...  
335        1
271        1
261        1
213        1
209        1
Name: EARNEDTIME, Length: 317, dtype: int64

## OREGON

In [13]:
QUERY = """
SELECT * FROM `recidiviz-staging.us_or_raw_data.RCDVZ_EDUTEST_NEEDCODEHISTORY1`
"""

or_edu_testing = pd.read_gbq(
    query=QUERY,
    project_id=PROJECT_ID,
    use_bqstorage_api=True,
    progress_bar_type="tqdm_notebook",
)

Downloading:   0%|          | 0/624278 [00:00<?, ?rows/s]

In [11]:
QUERY = """
SELECT * FROM `recidiviz-staging.us_or_raw_data.RCDVZ_EDUTEST_T_EDUCNEED`
"""

co_edu_2 = pd.read_gbq(
    query=QUERY,
    project_id=PROJECT_ID,
    use_bqstorage_api=True,
    progress_bar_type="tqdm_notebook",
)

Downloading:   0%|          | 0/12847 [00:00<?, ?rows/s]

# PERCENT SENTENCE SERVED

In [46]:
QUERY = """
SELECT 
    release_date,
    outflow_
    avg(sen.min_sentence_length_days_calculated),
    COUNT(*) AS supervision_population
FROM `sessions.compartment_sessions_materialized` s,
UNNEST(GENERATE_DATE_ARRAY('2015-01-01', DATE_TRUNC(CURRENT_DATE, YEAR), INTERVAL 1 YEAR)) as release_date

WHERE compartment_level_1 IN ('INCARCERATION')
    AND COALESCE(DATE_SUB(end_date_exclusive, INTERVAL 1 DAY), '9999-01-01')
        BETWEEN DATE_SUB(release_date, INTERVAL 1 YEAR) and release_date
    AND state_code = 'US_IX'
GROUP BY 1
ORDER BY 1
"""

sen_served = pd.read_gbq(
    query=QUERY,
    project_id=PROJECT_ID,
    use_bqstorage_api=True,
    progress_bar_type="tqdm_notebook",
)

GenericGBQException: Reason: 400 Unrecognized name: sen at [4:9]

Location: US
Job ID: 8906e63f-ef70-40f2-80bd-7d310b597100


In [44]:
sen_served

,release_date,supervision_population
0,2015-01-01,5890
1,2016-01-01,6424
2,2017-01-01,6248
3,2018-01-01,6656
4,2019-01-01,6684
5,2020-01-01,6344
6,2021-01-01,6885
7,2022-01-01,6016
8,2023-01-01,6296
9,2024-01-01,6940


In [37]:
QUERY = """
SELECT 
    *
FROM `sessions.compartment_sessions_materialized` ses
JOIN `state.state_person` person
    ON sen.person_id = person.person_id
JOIN `sessions.sentences_preprocessed_materialized` sen
LEFT JOIN `recidiviz-123.state.state_charge` charge
    ON sen.person_id = charge.person_id
    AND sen.charge_id = charge.charge_id
WHERE compartment_level_2 IN ('PROBATION')
    AND state_code = 'US_IX'
"""

df = pd.read_gbq(
    query=QUERY,
    project_id=PROJECT_ID,
    use_bqstorage_api=True,
    progress_bar_type="tqdm_notebook",
)

Downloading:   0%|          | 0/140845 [00:00<?, ?rows/s]

In [38]:
df.columns

Index(['person_id', 'session_id', 'sub_session_id_start', 'sub_session_id_end',
       'dataflow_session_id_start', 'dataflow_session_id_end', 'start_date',
       'end_date', 'end_date_exclusive', 'state_code', 'compartment_level_1',
       'compartment_level_2', 'session_length_days', 'session_days_inferred',
       'start_reason', 'start_sub_reason', 'end_reason',
       'is_inferred_start_reason', 'is_inferred_end_reason',
       'start_reason_original', 'end_reason_original', 'earliest_start_date',
       'last_day_of_data', 'inflow_from_level_1', 'inflow_from_level_2',
       'outflow_to_level_1', 'outflow_to_level_2', 'age_start', 'age_end',
       'gender', 'prioritized_race_or_ethnicity', 'assessment_score_start',
       'assessment_score_end', 'supervising_officer_external_id_start',
       'supervising_officer_external_id_end', 'compartment_location_start',
       'facility_name_start', 'supervision_office_name_start',
       'supervision_district_name_start', 'supervision_r